In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install necessary libraries if not already installed
!pip install pandas librosa scikit-learn

import os
import pandas as pd
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os

# Path to the base directory where your CSV data files are stored
base_path = '/content/drive/MyDrive/AI/Fumbling Model/fluent_speech_commands_dataset/data/'

# Load the CSV files into pandas dataframes
train_data = pd.read_csv(os.path.join(base_path, 'train_data.csv'))
test_data = pd.read_csv(os.path.join(base_path, 'test_data.csv'))
valid_data = pd.read_csv(os.path.join(base_path, 'valid_data.csv'))
speaker_demographics = pd.read_csv(os.path.join(base_path, 'speaker_demographics.csv'))

# Display the first few rows to verify the data
print("Train Data:")
print(train_data.head())

print("Test Data:")
print(test_data.head())

print("Validation Data:")
print(valid_data.head())

print("Speaker Demographics:")
print(speaker_demographics.head())


Train Data:
   Unnamed: 0                                               path  \
0           0  wavs/speakers/2BqVo8kVB2Skwgyb/0a3129c0-4474-1...   
1           1  wavs/speakers/2BqVo8kVB2Skwgyb/0ee42a80-4474-1...   
2           2  wavs/speakers/2BqVo8kVB2Skwgyb/144d5be0-4474-1...   
3           3  wavs/speakers/2BqVo8kVB2Skwgyb/1811b6e0-4474-1...   
4           4  wavs/speakers/2BqVo8kVB2Skwgyb/1d9f3920-4474-1...   

          speakerId          transcription           action  object location  
0  2BqVo8kVB2Skwgyb        Change language  change language    none     none  
1  2BqVo8kVB2Skwgyb                 Resume         activate   music     none  
2  2BqVo8kVB2Skwgyb     Turn the lights on         activate  lights     none  
3  2BqVo8kVB2Skwgyb   Switch on the lights         activate  lights     none  
4  2BqVo8kVB2Skwgyb  Switch off the lights       deactivate  lights     none  
Test Data:
   Unnamed: 0                                               path  \
0           0  wavs/speake

In [ ]:
import librosa
import numpy as np

# Function to extract MFCC features from the wav files
def extract_features(file_name):
    # Load the audio file
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

    # Extract MFCC features from the audio file
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

    # Compute the mean of the MFCC features across time axis
    mfccs_mean = np.mean(mfccs.T, axis=0)

    return mfccs_mean


In [ ]:
print("Columns in Train Data:")
print(train_data.columns)


Columns in Train Data:
Index(['Unnamed: 0', 'path', 'speakerId', 'transcription', 'action', 'object',
       'location'],
      dtype='object')


In [ ]:
print(train_data.columns)


Index(['Unnamed: 0', 'path', 'speakerId', 'transcription', 'action', 'object',
       'location'],
      dtype='object')


In [ ]:
import os
import pandas as pd
import librosa

def load_audio_files(base_path):
    features = []
    labels = []

    # Traverse through each speaker folder
    for speaker_folder in os.listdir(base_path):
        speaker_path = os.path.join(base_path, speaker_folder)

        if os.path.isdir(speaker_path):
            # Traverse through each audio file in the speaker folder
            for file_name in os.listdir(speaker_path):
                if file_name.endswith('.wav'):
                    file_path = os.path.join(speaker_path, file_name)
                    try:
                        # Load the audio file
                        audio, sr = librosa.load(file_path, sr=None)
                        # Extract features (for example, MFCCs)
                        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
                        mfccs_mean = mfccs.mean(axis=1)
                        features.append(mfccs_mean)
                        labels.append(speaker_folder)  # Use folder name as label
                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

    return features, labels

# Define the path to the speakers directory
base_path = '/content/drive/MyDrive/AI/Fumbling Model/fluent_speech_commands_dataset/wavs/speakers'

# Load audio files
train_features, train_labels = load_audio_files(base_path)

# Convert to DataFrame for easier handling
train_data = pd.DataFrame(train_features)
train_data['label'] = train_labels

print(f'Train Features Shape: {train_data.shape}')
print(f'Train Labels: {train_data["label"].unique()}')


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

# Step 1: Encode the labels into numerical form
le = LabelEncoder()
train_labels_encoded = le.fit_transform(train_data['label'])

# Step 2: Split the data into training and validation sets
train_features, valid_features, train_labels, valid_labels = train_test_split(
    train_data.drop('label', axis=1),
    train_labels_encoded,
    test_size=0.2,  # 20% for validation
    random_state=42
)

# Step 3: Train an SVM classifier
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(train_features, train_labels)

# Validate the model on validation data
valid_accuracy = svm_model.score(valid_features, valid_labels)
print(f'Validation Accuracy: {valid_accuracy:.2f}')


Validation Accuracy: 0.94


In [ ]:
# Assuming your labels are encoded as follows:
# 0 = Unconfident, 1 = Confident
# Adjust the following lines accordingly if the mapping is different.

# Load your pre-trained SVM model and label encoder
# from joblib import load
# svm_model = load('path_to_your_model.pkl')
# le = load('path_to_label_encoder.pkl')

# Upload the audio file
from google.colab import files
uploaded = files.upload()

# Assuming the uploaded file is the only file and it's in WAV format
for file_name in uploaded.keys():
    audio_file_path = file_name
    features = extract_features(audio_file_path)

    if features is not None:
        features = features.reshape(1, -1)  # Reshape for prediction
        prediction = svm_model.predict(features)
        confidence = svm_model.predict_proba(features)

        # Probability of being confident (1)
        confidence_percentage = confidence[0][1] * 100  # Adjust index based on your encoding
        # fumbleness_percentage = 100 - confidence_percentage / # Fumbleness is 100% - confidence

        # Determine confidence based on fumbleness percentage
        if confidence_percentage > 75:
            confidence_label = "Unconfident"
        else:
            confidence_label = "Confident"

        print(f'Predicted Class: {le.inverse_transform(prediction)[0]}')
        print(f'Fumbleness Percentage: {fumbleness_percentage:.2f}%')
        print(f'Confidence Status: {confidence_label}')
    else:
        print('Could not extract features. Please check the audio file.')


Saving 029f6450-447a-11e9-a9a5-5dbec3b8816a.wav to 029f6450-447a-11e9-a9a5-5dbec3b8816a (1).wav
Predicted Class: 2BqVo8kVB2Skwgyb
Fumbleness Percentage: 99.74%
Confidence Status: Confident
